In [ ]:

import numpy as np
import pandas as pd
import warnings
import neurom as nm
from neurom.apps import morph_stats
from sklearn.model_selection import cross_validate, ShuffleSplit
from xgboost import XGBClassifier
import yaml

# From the repo
from data.IN_pwd import IN_pwd_list

cv = ShuffleSplit(n_splits=50, test_size=.20, random_state=0)

final_results = {}
for layer in IN_pwd_list.keys():
    dataset_list = []
    i = 0
    for population in IN_pwd_list[layer]:
        neurons = nm.load_morphologies(population)
        warnings.filterwarnings("ignore", category=UserWarning)
        with open('../data/morphometrics_config_file.yaml') as mcf:
            morphometrics_config = yaml.safe_load(mcf)
        df_metrics = morph_stats.extract_dataframe(morphs = neurons, config = morphometrics_config)
        df_metrics.iloc[:,0] = i
        i += 1
        dataset_list.append(df_metrics)
    dataset = pd.concat(dataset_list)
    data_nonan = dataset.loc[:, dataset.isna().sum() == 0]
    data = np.array(data_nonan)
    np.random.shuffle(data)
    X, y = data[:,1:], list(data[:,0])
    score = cross_validate(XGBClassifier(), X, y, cv = cv, return_train_score = True)
    test_score = score['test_score']
    #train_score = score['train_score']
    #fit_time = np.mean(score['fit_time'])
    final_results['xgb_morpho_'+ layer] = np.mean(test_score)

In [ ]:
import pickle

# save dictionary to xgb_morpho_results.pkl file
with open('../results/IN_results/xgb_morpho_results.pkl', 'wb') as fp:
    pickle.dump(final_results, fp)
    print('dictionary saved successfully to file')